<a href="https://colab.research.google.com/github/vishnuvardhanreddyav/Data_sci/blob/main/Anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install plotly

In [2]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.0 MB/s 
     |████████████████████████████████| 2.0 MB 31.6 MB/s 
     |████████████████████████████████| 1.7 MB 42.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 47.8 MB/s 
     |████████████████████████████████| 4.8 MB 45.2 MB/s 
     |████████████████████████████████| 6.8 MB 39.8 MB/s 
     |████████████████████████████████| 17.0 MB 38.7 MB/s 
     |████████████████████████████████| 137 kB 56.4 MB/s 
     |████████████████████████████████| 25.9 MB 1.3 MB/s 
     |████████████████████████████████| 167 kB 45.2 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 88 kB 7.5 MB/s 
     |████████████████████████████████| 

In [11]:
import pandas as pd
import plotly.express as px
from pycaret.anomaly import *

In [12]:
data = pd.read_csv('/content/machine_temperature_system_failure.csv')
data.head()

,timestamp,value
0,2013-12-02 21:15:00,73.967322
1,2013-12-02 21:20:00,74.935882
2,2013-12-02 21:25:00,76.124162
3,2013-12-02 21:30:00,78.140707
4,2013-12-02 21:35:00,79.329836


In [13]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.head()

,timestamp,value
0,2013-12-02 21:15:00,73.967322
1,2013-12-02 21:20:00,74.935882
2,2013-12-02 21:25:00,76.124162
3,2013-12-02 21:30:00,78.140707
4,2013-12-02 21:35:00,79.329836


In [14]:
data.isnull().sum()

timestamp    0
value        0
dtype: int64

In [15]:
data.describe()

,value
count,22695.000000
mean,85.926498
std,13.746912
min,2.084721
25%,83.080078
50%,89.408246
75%,94.016252
max,108.510543


In [16]:
data.head(5)
data.head(-5)

,timestamp,value
0,2013-12-02 21:15:00,73.967322
1,2013-12-02 21:20:00,74.935882
2,2013-12-02 21:25:00,76.124162
3,2013-12-02 21:30:00,78.140707
4,2013-12-02 21:35:00,79.329836
...,...,...
22685,2014-02-19 14:40:00,96.739868
22686,2014-02-19 14:45:00,97.284578
22687,2014-02-19 14:50:00,97.549774
22688,2014-02-19 14:55:00,98.162952


In [17]:
# create moving-averages
data['MA48'] = data['value'].rolling(48).mean()
data['MA336'] = data['value'].rolling(336).mean()


In [18]:
# plot 
import plotly.express as px
fig = px.line(data, x="timestamp", y=['value', 'MA48', 'MA336'], title='Temperature', template = 'plotly_dark')
fig.show()

In [19]:
# drop moving-average columns
data.drop(['MA48', 'MA336'], axis=1, inplace=True)
# set timestamp to index
data.set_index('timestamp', drop=True, inplace=True)
# resample timeseries to hourly 
data = data.resample('H').sum()


In [20]:
# creature features from date
data['day'] = [i.day for i in data.index]
data['day_name'] = [i.day_name() for i in data.index]
data['day_of_year'] = [i.dayofyear for i in data.index]
data['week_of_year'] = [i.weekofyear for i in data.index]
data['hour'] = [i.hour for i in data.index]
data['is_weekday'] = [i.isoweekday() for i in data.index]
data.head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday
timestamp,,,,,,,
2013-12-02 21:00:00,702.104364,2,Monday,336,49,21,1
2013-12-02 22:00:00,967.176148,2,Monday,336,49,22,1
2013-12-02 23:00:00,979.500222,2,Monday,336,49,23,1
2013-12-03 00:00:00,995.585452,3,Tuesday,337,49,0,2
2013-12-03 01:00:00,1014.349168,3,Tuesday,337,49,1,2


In [21]:
# init setup
from pycaret.anomaly import *


In [22]:
s = setup(data, session_id = 123)

,Description,Value
0,session_id,123
1,Original Data,"(1891, 7)"
2,Missing Values,False
3,Numeric Features,4
4,Categorical Features,3
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(1891, 30)"
9,CPU Jobs,-1


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=[],
                                      target='UNSUPERVISED_DUMMY_TARGET',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='most frequent',
                                fill_value_categorical=None,
                                fill_value_numerical=None...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
        

In [23]:
# check list of available models
models()

INFO:logs:gpu_param set to False


,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [24]:
# train model
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:IForest(behaviour='new', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=-1, random_state=123,
    verbose=0)
INFO:logs:create_model() succesfully completed......................................
INFO:logs:Initializing assign_model()
INFO:logs:assign_model(model=IForest(behaviour='new', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=-1, random_state=123,
    verbose=0), transformation=False, score=True, verbose=True)
INFO:logs:Checking exceptions
INFO:logs:Determining Trained Model
INFO:logs:Trained Model : Isolation Forest
INFO:logs:Copying data
INFO:logs:(1891, 9)
INFO:logs:assign_model() succesfully completed......................................


,value,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
timestamp,,,,,,,,,
2013-12-02 21:00:00,702.104364,2,Monday,336,49,21,1,1,0.004447
2013-12-02 22:00:00,967.176148,2,Monday,336,49,22,1,0,-0.003893
2013-12-02 23:00:00,979.500222,2,Monday,336,49,23,1,0,-0.001969
2013-12-03 00:00:00,995.585452,3,Tuesday,337,49,0,2,0,-0.015928
2013-12-03 01:00:00,1014.349168,3,Tuesday,337,49,1,2,0,-0.022514


In [25]:

plot_model(iforest, plot = 'umap')

INFO:logs:Rendering Visual


INFO:logs:Visual Rendered Successfully
INFO:logs:plot_model() succesfully completed......................................


In [26]:
# check anomalies
iforest_results[iforest_results['Anomaly'] == 1].head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
timestamp,,,,,,,,,
2013-12-02 21:00:00,702.104364,2,Monday,336,49,21,1,1,0.004447
2013-12-05 16:00:00,733.802380,5,Thursday,339,49,16,4,1,0.003019
2013-12-05 17:00:00,710.312627,5,Thursday,339,49,17,4,1,0.006333
2013-12-05 18:00:00,686.686472,5,Thursday,339,49,18,4,1,0.010184
2013-12-05 19:00:00,660.694548,5,Thursday,339,49,19,4,1,0.016097


In [28]:
import plotly.graph_objects as go
# plot value on y-axis and date on x-axis
fig = px.line(iforest_results, x=iforest_results.index, y="value", title='Temperature Readings - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [iforest_results.loc[i]['value'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()

In [29]:
# train model
knn = create_model('knn', fraction = 0.1)
knn_results = assign_model(iforest)
knn_results.head()

INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
  radius=1.0)
INFO:logs:create_model() succesfully completed......................................
INFO:logs:Initializing assign_model()
INFO:logs:assign_model(model=IForest(behaviour='new', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=-1, random_state=123,
    verbose=0), transformation=False, score=True, verbose=True)
INFO:logs:Checking exceptions
INFO:logs:Determining Trained Model
INFO:logs:Trained Model : Isolation Forest
INFO:logs:Copying data
INFO:logs:(1891, 9)
INFO:logs:assign_model() succesfully completed......................................


,value,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
timestamp,,,,,,,,,
2013-12-02 21:00:00,702.104364,2,Monday,336,49,21,1,1,0.004447
2013-12-02 22:00:00,967.176148,2,Monday,336,49,22,1,0,-0.003893
2013-12-02 23:00:00,979.500222,2,Monday,336,49,23,1,0,-0.001969
2013-12-03 00:00:00,995.585452,3,Tuesday,337,49,0,2,0,-0.015928
2013-12-03 01:00:00,1014.349168,3,Tuesday,337,49,1,2,0,-0.022514


In [30]:
plot_model(knn, plot = 'umap')

INFO:logs:Rendering Visual


INFO:logs:Visual Rendered Successfully
INFO:logs:plot_model() succesfully completed......................................


In [31]:
knn_results[knn_results['Anomaly'] == 1].head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
timestamp,,,,,,,,,
2013-12-02 21:00:00,702.104364,2,Monday,336,49,21,1,1,0.004447
2013-12-05 16:00:00,733.802380,5,Thursday,339,49,16,4,1,0.003019
2013-12-05 17:00:00,710.312627,5,Thursday,339,49,17,4,1,0.006333
2013-12-05 18:00:00,686.686472,5,Thursday,339,49,18,4,1,0.010184
2013-12-05 19:00:00,660.694548,5,Thursday,339,49,19,4,1,0.016097


In [33]:

# plot value on y-axis and date on x-axis
fig_knn = px.line(knn_results, x=knn_results.index, y="value", title='Temperature Readings - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates_knn = knn_results[knn_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values_knn = [knn_results.loc[i]['value'] for i in outlier_dates]
fig_knn.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig_knn.show()